In [1]:
import numpy as np


# 수치미분 함수
def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad


# sigmoid 함수
def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
# LogicGate Class

class LogicGate:
    
    def __init__(self, gate_name, xdata, tdata, lr, cnt):  # xdata, tdata => numpy.array(...)
        
        self.name = gate_name
        
        self.xdata = xdata.reshape(4, 2)
        self.tdata = tdata.reshape(4, 1)
        
        # 가중치 W, 바이어스 b 초기화 - 일반적인 방법으로 변경 필요
        self.W = np.random.rand(self.xdata.shape[1], 1)  # weight, 2 X 1 matrix
        self.b = np.random.rand(1)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = lr
        
        # 반복횟수 초기화
        self.iteration_count = cnt
        
    # 손실함수
    def loss_func(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.xdata, self.W) + self.b
        y = sigmoid(z)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )      
    
    # 손실 값 계산
    def error_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.xdata, self.W) + self.b
        y = sigmoid(z)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )

    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self):
        
        step_count = int(self.iteration_count * 0.1)
        
        f = lambda x : self.loss_func()
        
        print("Initial error value = ", self.error_val())
        
        for step in  range(self.iteration_count):
            
            self.W -= self.learning_rate * numerical_derivative(f, self.W)
    
            self.b -= self.learning_rate * numerical_derivative(f, self.b)
    
            if (step % step_count == 0):
                print("step = ", step, "error value = ", self.error_val())
                
                
    # 미래 값 예측 함수
    def predict(self, input_data):
        
        z = np.dot(input_data, self.W) + self.b
        y = sigmoid(z)
    
        if y > 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result
    
    # 정확도 예측 함수
    def accuracy(self, test_xdata, test_tdata):
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(xdata)):
            
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        print("Accuracy => ", len(matched_list) / len(test_xdata) )
        
        return matched_list, not_matched_list

In [3]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 0, 0, 1])

AND_obj = LogicGate("AND_GATE", xdata, tdata, 1e-2, 20001)

AND_obj.train()

Initial error value =  4.584819352723884
step =  0 error value =  4.527975978254977
step =  2000 error value =  0.6691143102024688
step =  4000 error value =  0.3934274366158953
step =  6000 error value =  0.2765028599798748
step =  8000 error value =  0.21221069528150846
step =  10000 error value =  0.17175078780127748
step =  12000 error value =  0.14403527912211944
step =  14000 error value =  0.12390482339090012
step =  16000 error value =  0.10864222930908682
step =  18000 error value =  0.0966841706119444
step =  20000 error value =  0.08706918688108398


In [5]:
# AND Gate prediction
print(AND_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 0, 0, 1])

(true_list, not_true_list) = AND_obj.accuracy(test_xdata, test_tdata)

print("True List => ", true_list, ", Not True List => ", not_true_list)

AND_GATE 

Accuracy =>  1.0
True List =>  [0, 1, 2, 3] , Not True List =>  []


In [6]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 1])

OR_obj = LogicGate("OR_GATE", xdata, tdata, 1e-2, 20001)

OR_obj.train() 

Initial error value =  1.834506576886361
step =  0 error value =  1.8320051682120388
step =  2000 error value =  0.4326416620535259
step =  4000 error value =  0.23288090424498034
step =  6000 error value =  0.15723250782571185
step =  8000 error value =  0.11809936346145677
step =  10000 error value =  0.09434599921055167
step =  12000 error value =  0.07844924396095036
step =  14000 error value =  0.06708642074219566
step =  16000 error value =  0.05857022571862583
step =  18000 error value =  0.05195532956507884
step =  20000 error value =  0.04667190476274162


In [7]:
# OR Gate prediction
print(OR_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 1])

(true_list, not_true_list) = OR_obj.accuracy(test_xdata, test_tdata)

print("True List => ", true_list, ", Not True List => ", not_true_list)

OR_GATE 

Accuracy =>  1.0
True List =>  [0, 1, 2, 3] , Not True List =>  []


In [8]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([1, 1, 1, 0])

NAND_obj = LogicGate("NAND_GATE", xdata, tdata, 1e-2, 20001)

NAND_obj.train()

Initial error value =  2.612814123368244
step =  0 error value =  2.607307733175956
step =  2000 error value =  0.6693363853526373
step =  4000 error value =  0.39350380848010574
step =  6000 error value =  0.27654144673824743
step =  8000 error value =  0.21223378181464336
step =  10000 error value =  0.17176607449438935
step =  12000 error value =  0.14404611442655774
step =  14000 error value =  0.1239128887514547
step =  16000 error value =  0.10864845831423378
step =  18000 error value =  0.09668912175107933
step =  20000 error value =  0.0870732140942983


In [9]:
# NAND Gate prediction
print(NAND_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 1, 1, 1, 0])

(true_list, not_true_list) = NAND_obj.accuracy(test_xdata, test_tdata)

print("True List => ", true_list, ", Not True List => ", not_true_list)

NAND_GATE 

Accuracy =>  1.0
True List =>  [0, 1, 2, 3] , Not True List =>  []


In [10]:
# XOR 을 NAND + OR => AND 조합으로 계산함
input_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

s1 = []    # NAND 출력
s2 = []    # OR 출력

new_input_data = []  # AND 입력
final_output = []    # AND 출력

for index in range(len(input_data)):
    
    s1 = NAND_obj.predict(input_data[index])  # NAND 출력
    s2 = OR_obj.predict(input_data[index])    # OR 출력
    
    new_input_data.append(s1[-1])    # AND 입력
    new_input_data.append(s2[-1])    # AND 입력
    
    (sigmoid_val, logical_val) = AND_obj.predict(np.array(new_input_data))
    
    final_output.append(logical_val)    # AND 출력, 즉 XOR 출력    
    new_input_data = []    # AND 입력 초기화


for index in range(len(input_data)):    
    print(input_data[index], " = ", final_output[index], end='')
    print("\n")

[0 0]  =  0

[0 1]  =  1

[1 0]  =  1

[1 1]  =  0

